In [1]:
import numpy as np
import pandas as pd

In [2]:
def load_and_normalize(path):
    """Load a CSV, strip and lowercase its column names."""
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    return df


# NGS chunks for 2016 & 2017
ngs_paths = [
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2016-pre.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2016-post.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk1-6.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk7-12.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk13-17.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2017-pre.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2017-post.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2017-reg-wk1-6.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2017-reg-wk7-12.csv',
    '/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/NGS-2017-reg-wk13-17.csv',
]
df = pd.concat([load_and_normalize(p) for p in ngs_paths], ignore_index=True)

ndtypes = {'gamekey': 'int16',         
           'playid': 'int16',         
           'gsisid': 'float32',        
           'time': 'str',         
           'x': 'float32',         
           'y': 'float32',         
           'dis': 'float32',
           'o': 'float32',
           'event': 'str'}

df = df.astype(ndtypes)
df['gsisid'] = df['gsisid'].fillna(-1)

players = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/play_player_role_data.csv')
revs = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/video_review.csv', 
                   usecols=['Season_Year', 'GameKey', 'PlayID', 'GSISID', 'Primary_Partner_GSISID'],
                   na_values=['Unclear']).fillna(-99).astype(int)

/var/folders/4k/nl2zz_ln4d55dcscl_vvjglh0000gn/T/ipykernel_7845/618863554.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [5]:
players = players.merge(revs, left_on=['Season_Year', 'GameKey', 'PlayID', 'GSISID'], right_on=['Season_Year', 'GameKey', 'PlayID', 'GSISID'], how='left',
                        suffixes=('', '_dupe'), sort=False)

In [6]:
players['concussed'] = np.where(players['Primary_Partner_GSISID_dupe'].isnull(), 0, 1)

In [12]:
playas = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/player_punt_data.csv')
playas_agg = playas.groupby('GSISID')['Number'].agg(' '.join).to_frame()
players = players.merge(playas_agg, on='GSISID', how='left')

In [13]:
drops = ['Primary_Partner_GSISID'] + players.columns[players.columns.str.contains('dupe')].tolist()

In [15]:
players = players.drop(drops, axis=1).sort_values(['GSISID', 'GameKey', 'PlayID']).set_index('GSISID').reset_index()

In [69]:
plays = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/play_information.csv', index_col=['GameKey', 'PlayID'])
games = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/game_data.csv', index_col='GameKey')
games['Tempearature'] = games['Temperature'].fillna(-999)
plays_all = plays.join(games, rsuffix='_dupe', sort=False)

revs = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/video_review.csv', index_col=['GameKey', 'PlayID'])
revs['Primary_Partner_GSISID'] = revs['Primary_Partner_GSISID'].replace('Unclear', np.nan)
revs['Primary_Partner_GSISID'] = pd.to_numeric(revs.Primary_Partner_GSISID)
plays_all = plays_all.join(revs, rsuffix='_dupe2', sort=False)

playernums = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/player_punt_data.csv')
playernums = playernums.groupby('GSISID')['Number'].agg(' '.join).to_frame()

plays_all = plays_all.reset_index().merge(playernums, how='left', on='GSISID', sort=False)

roles = pd.read_csv('/Users/petershmorhun/Documents/GitHub/summer_milestone_2/datasets/NFL-Punt-Analytics-Competition/play_player_role_data.csv')
roles_all = roles

plays_all = plays_all.merge(playernums, how='left', 
            left_on='Primary_Partner_GSISID', right_on='GSISID', 
            suffixes=("_player", "_partner"), sort=False)

plays_all = plays_all.merge(roles_all, how='left', left_on=['GameKey', 
            'PlayID', 'Primary_Partner_GSISID'], right_on=['GameKey', 
            'PlayID', 'GSISID'], suffixes=("_player", "_partner"), sort=False)

plays_all.set_index(['GameKey', 'PlayID'], inplace=True)

roles_enc = pd.get_dummies(roles_all, columns=['Role'], prefix='Role', dtype=int)
# This would keep all original columns including the ones you need to group by
collist = list(roles_enc)[2:]
agglist = ['size', pd.Series.nunique] + (len(collist)-3) * ['sum']
aggdict = dict(zip(collist, agglist))
roles_agg = roles_enc.groupby(['Season_Year', 'GameKey', 'PlayID']).agg(aggdict)
plays_all = plays_all.join(roles_agg, rsuffix="_roles")

#%% make simple features
plays_all['yard_number'] = plays_all.YardLine.str.split().str[1].astype(int)
plays_all['dist_togoal'] = np.where(plays_all.Poss_Team == plays_all.YardLine\
            .str.split().str[0], plays_all.yard_number + 50, 
            plays_all.yard_number)
plays_all['Rec_team'] = np.where(plays_all.Poss_Team == plays_all.HomeTeamCode, 
             plays_all.VisitTeamCode, plays_all.HomeTeamCode)
plays_all['home_score'] = plays_all.Score_Home_Visiting.str.split(" - ")\
            .str[0].astype(int)
plays_all['visit_score'] = plays_all.Score_Home_Visiting.str.split(" - ")\
            .str[1].astype(int)
plays_all['concussion'] = np.where(plays_all.Primary_Impact_Type.isnull(), 
                                    0, 1)

#%% clean up 
drops = ['YardLine',
         'Play_Type',
         'Home_Team_Visit_Team',
         'Primary_Partner_GSISID',
         'Score_Home_Visiting']\
         + plays_all.columns[plays_all.columns.str.contains('dupe')].tolist()
plays_all.drop(drops, axis=1, inplace=True)

plays_all['GSISID_player'] = plays_all.GSISID_player.fillna(-99, 
                                downcast='infer')
plays_all['GSISID_partner'] = plays_all.GSISID_partner.fillna(-99, 
                                downcast='infer')

floatcols = plays_all.select_dtypes('float').columns
for f in floatcols:
    plays_all[f] = plays_all[f].fillna(-99).astype(int)

plays_all.fillna('unspecified', inplace=True)
plays_all.replace('SD', 'LAC', inplace=True, regex=True)
plays_all['Game_Date'] = pd.to_datetime(plays_all.Game_Date, format='%m/%d/%Y')

plays_all.sort_index(inplace=True)

/var/folders/4k/nl2zz_ln4d55dcscl_vvjglh0000gn/T/ipykernel_7845/4171865530.py:60: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  plays_all['GSISID_player'] = plays_all.GSISID_player.fillna(-99,
/var/folders/4k/nl2zz_ln4d55dcscl_vvjglh0000gn/T/ipykernel_7845/4171865530.py:62: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  plays_all['GSISID_partner'] = plays_all.GSISID_partner.fillna(-99,


In [64]:
plays_all = plays_all.join(roles_agg, rsuffix="_roles")

In [77]:
plays_all

Season_Year_player Season_Type  Game_Date  Week  \
GameKey PlayID Season_Year                                                    
2       191    2016                       2016         Pre 2016-08-13     2   
        1132   2016                       2016         Pre 2016-08-13     2   
        1227   2016                       2016         Pre 2016-08-13     2   
        1864   2016                       2016         Pre 2016-08-13     2   
        2247   2016                       2016         Pre 2016-08-13     2   
...                                        ...         ...        ...   ...   
664     1046   2017                       2017        Post 2018-01-21     3   
        1206   2017                       2017        Post 2018-01-21     3   
        1854   2017                       2017        Post 2018-01-21     3   
        3168   2017                       2017        Post 2018-01-21     3   
665     1094   2017                       2017        Post 2018-02-04     4   

                           Game_Clock  Quarter Poss_Team  \
GameKey PlayID Season_Year                                 
2       191    2016             12:30        1        LA   
        1132   2016             12:08        2        LA   
        1227   2016             10:01        2       DAL   
        1864   2016             00:21        2        LA   
        2247   2016             10:26        3       DAL   
...                               ...      ...       ...   
664     1046   2017             12:07        2       MIN   
        1206   2017             09:27        2       PHI   
        1854   2017             00:38        2       MIN   
        3168   2017             09:45        4       PHI   
665     1094   2017             13:16        2       PHI   

                                                              PlayDescription  \
GameKey PlayID Season_Year                                                      
2       191    2016         (12:30) J.Hekker punts 52 yards to DAL 1, Cent...   
        1132   2016         (12:08) J.Hekker punts 51 yards to DAL 20, Cen...   
        1227   2016         (10:01) C.Jones punts 40 yards to LA 42, Cente...   
        1864   2016         (:21) J.Hekker punts 31 yards to DAL 15, Cente...   
        2247   2016         (10:26) M.Wile punts 40 yards to LA 45, Center...   
...                                                                       ...   
664     1046   2017         (12:07) R.Quigley punts 47 yards to PHI 20, Ce...   
        1206   2017         (9:27) D.Jones punts 41 yards to MIN 15, Cente...   
        1854   2017         (:38) R.Quigley punts 53 yards to end zone, Ce...   
        3168   2017         (9:45) D.Jones punts 45 yards to MIN 10, Cente...   
665     1094   2017         (13:16) D.Jones punts 41 yards to NE 37, Cente...   

                            Game_Day     Game_Site  ... Role_VLi Role_VLo  \
GameKey PlayID Season_Year                          ...                     
2       191    2016         Saturday   Los Angeles  ...        0        0   
        1132   2016         Saturday   Los Angeles  ...        1        1   
        1227   2016         Saturday   Los Angeles  ...        1        1   
        1864   2016         Saturday   Los Angeles  ...        0        0   
        2247   2016         Saturday   Los Angeles  ...        1        1   
...                              ...           ...  ...      ...      ...   
664     1046   2017           Sunday  Philadelphia  ...        0        0   
        1206   2017           Sunday  Philadelphia  ...        1        1   
        1854   2017           Sunday  Philadelphia  ...        0        0   
        3168   2017           Sunday  Philadelphia  ...        0        0   
665     1094   2017           Sunday   Minneapolis  ...        0        0   

                           Role_VR Role_VRi yard_number dist_togoal Rec_team  \
GameKey PlayID Season_Year                                                     
2  

In [78]:
plays_all = plays_all.drop(['PlayID'], axis=1)  # Drop the duplicate columns

In [82]:
players.head()

,GSISID,Season_Year,GameKey,PlayID,Role,concussed,Number
0,19714,2016,7,242,P,0,9
1,19714,2016,7,817,P,0,9
2,19714,2016,7,1800,P,0,9
3,19714,2016,7,2067,P,0,9
4,19714,2016,7,2410,P,0,9


In [79]:
plays_all_reset = plays_all.reset_index()
# Now both have GameKey, PlayID as regular columns

In [80]:
plays_all_reset

,GameKey,PlayID,Season_Year,Season_Year_player,Season_Type,Game_Date,Week,Game_Clock,Quarter,Poss_Team,...,Role_VLi,Role_VLo,Role_VR,Role_VRi,yard_number,dist_togoal,Rec_team,home_score,visit_score,concussion
0,2,191,2016.0,2016,Pre,2016-08-13,2,12:30,1,LA,...,0,0,1,0,47,97,DAL,0,7,0
1,2,1132,2016.0,2016,Pre,2016-08-13,2,12:08,2,LA,...,1,1,0,1,29,79,DAL,7,21,0
2,2,1227,2016.0,2016,Pre,2016-08-13,2,10:01,2,DAL,...,1,1,0,1,18,68,LA,7,21,0
3,2,1864,2016.0,2016,Pre,2016-08-13,2,00:21,2,LA,...,0,0,0,0,46,46,DAL,7,24,0
4,2,2247,2016.0,2016,Pre,2016-08-13,2,10:26,3,DAL,...,1,1,0,1,15,65,LA,14,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,664,1046,2017.0,2017,Post,2018-01-21,3,12:07,2,MIN,...,0,0,1,0,33,83,PHI,14,7,0
6677,664,1206,2017.0,2017,Post,2018-01-21,3,09:27,2,PHI,...,1,1,1,0,44,94,MIN,14,7,0
6678,664,1854,2017.0,2017,Post,2018-01-21,3,00:38,2,MIN,...,0,0,1,0,47,97,PHI,21,7,0
6679,664,3168,2017.0,2017,Post,2018-01-21,3,09:45,4,PHI,...,0,0,1,0,45,95,MIN,38,7,0


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60716164 entries, 0 to 60716163
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   season_year  int64  
 1   gamekey      int16  
 2   playid       int16  
 3   gsisid       float32
 4   time         object 
 5   x            float32
 6   y            float32
 7   dis          float32
 8   o            float32
 9   dir          float64
 10  event        object 
dtypes: float32(5), float64(1), int16(2), int64(1), object(2)
memory usage: 3.2+ GB


In [93]:
motion_agg = df.groupby(['season_year', 'gamekey', 'playid', 'gsisid']).agg({
    'dis': ['sum', 'mean', 'max', 'std'],
    'x': ['min', 'max', lambda x: x.max() - x.min()],
    'y': ['min', 'max', lambda x: x.max() - x.min()],
})

# Give the lambda a proper name
motion_agg.columns = ['_'.join(col) if col[1] != '<lambda_0>' 
                     else col[0] + '_range' 
                     for col in motion_agg.columns]

In [95]:
motion_agg = motion_agg.reset_index()

In [96]:
motion_agg

,season_year,gamekey,playid,gsisid,dis_sum,dis_mean,dis_max,dis_std,x_min,x_max,x_range,y_min,y_max,y_range
0,2016,4,291,24061.0,67.629997,0.158384,0.73,0.149915,79.379997,103.699997,24.320000,26.559999,50.779999,24.219999
1,2016,4,291,25580.0,17.580000,0.112692,0.19,0.052834,40.099998,57.610001,17.510002,-0.500000,4.670000,5.170000
2,2016,4,291,26275.0,5.760000,0.144000,0.23,0.032249,46.150002,51.680000,5.529999,-0.440000,2.180000,2.620000
3,2016,4,291,26321.0,25.760000,0.207742,0.34,0.056336,49.049999,61.040001,11.990002,44.209999,53.700001,9.490002
4,2016,4,291,26664.0,40.639999,0.095176,0.29,0.072288,47.119999,57.650002,10.530003,2.670000,31.209999,28.539999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274211,2017,665,1094,33555.0,87.879997,0.198824,0.91,0.226803,47.750000,89.720001,41.970001,7.040000,46.180000,39.140000
274212,2017,665,1094,33573.0,83.080002,0.187964,1.01,0.279072,46.689999,89.629997,42.939999,-0.130000,25.580000,25.710000
274213,2017,665,1094,33577.0,85.360001,0.193122,0.86,0.235640,60.360001,99.570000,39.209999,13.010000,39.450001,26.440001
274214,2017,665,1094,33725.0,65.639999,0.148507,0.75,0.184189,64.699997,100.099998,35.400002,-0.190000,29.059999,29.249999


In [98]:
players.columns = players.columns.str.lower()

In [104]:
players['concussed'].value_counts()

concussed
0    146536
1        37
Name: count, dtype: int64

In [106]:
combined = players.merge(motion_agg, left_on=['season_year', 'gamekey', 'playid', 'gsisid'], right_on=['season_year', 'gamekey', 'playid', 'gsisid'], how='left', sort=False)

In [109]:
combined.query('concussed == 1')

,gsisid,season_year,gamekey,playid,role,concussed,number,dis_sum,dis_mean,dis_max,dis_std,x_min,x_max,x_range,y_min,y_max,y_range
2163,23564,2016,266,2902,PR,1,43,39.779999,0.054344,0.66,0.116853,85.330002,92.209999,6.879997,12.200000,27.780001,15.580001
2317,23742,2016,274,3609,P,1,9,69.500000,0.187838,0.95,0.285784,21.500000,73.690002,52.190002,19.570000,29.639999,10.070000
5259,26035,2017,399,3312,PR,1,80,73.669998,0.104645,0.60,0.089048,51.840000,74.370003,22.530003,19.020000,39.360001,20.340000
8475,27060,2017,506,1988,PR,1,21,50.290001,0.110771,1.16,0.137303,33.990002,44.700001,10.709999,30.150000,49.790001,19.640001
12180,27595,2016,189,3509,PR,1,81,37.939999,0.210778,0.68,0.207987,40.400002,49.810001,9.410000,6.320000,22.570000,16.250000
12618,27654,2016,280,3746,GL,1,17,71.379997,0.213713,0.95,0.289087,21.180000,65.139999,43.959999,25.500000,50.630001,25.130001
19606,28128,2016,149,3663,PFB,1,42,79.010002,0.235851,0.53,0.143134,52.990002,89.790001,36.799999,19.360001,51.439999,32.079998
23209,28620,2016,234,3278,PRG,1,94,70.330002,0.206853,0.90,0.304514,9.970000,61.689999,51.719998,30.590000,54.470001,23.880001
26905,28987,2016,218,3468,PLG,1,89,47.759998,0.207652,0.87,0.270640,64.070000,97.559998,33.489998,25.209999,35.090000,9.880001
26918,28987,2016,281,1526,PLG,1,89,46.549999,0.282121,0.88,0.312105,16.280001,54.619999,38.339998,30.600000,37.299999,6.699999


In [111]:
combined.query('concussed == 1')

,gsisid,season_year,gamekey,playid,role,concussed,number,dis_sum,dis_mean,dis_max,dis_std,x_min,x_max,x_range,y_min,y_max,y_range
2163,23564,2016,266,2902,PR,1,43,39.779999,0.054344,0.66,0.116853,85.330002,92.209999,6.879997,12.200000,27.780001,15.580001
2317,23742,2016,274,3609,P,1,9,69.500000,0.187838,0.95,0.285784,21.500000,73.690002,52.190002,19.570000,29.639999,10.070000
5259,26035,2017,399,3312,PR,1,80,73.669998,0.104645,0.60,0.089048,51.840000,74.370003,22.530003,19.020000,39.360001,20.340000
8475,27060,2017,506,1988,PR,1,21,50.290001,0.110771,1.16,0.137303,33.990002,44.700001,10.709999,30.150000,49.790001,19.640001
12180,27595,2016,189,3509,PR,1,81,37.939999,0.210778,0.68,0.207987,40.400002,49.810001,9.410000,6.320000,22.570000,16.250000
12618,27654,2016,280,3746,GL,1,17,71.379997,0.213713,0.95,0.289087,21.180000,65.139999,43.959999,25.500000,50.630001,25.130001
19606,28128,2016,149,3663,PFB,1,42,79.010002,0.235851,0.53,0.143134,52.990002,89.790001,36.799999,19.360001,51.439999,32.079998
23209,28620,2016,234,3278,PRG,1,94,70.330002,0.206853,0.90,0.304514,9.970000,61.689999,51.719998,30.590000,54.470001,23.880001
26905,28987,2016,218,3468,PLG,1,89,47.759998,0.207652,0.87,0.270640,64.070000,97.559998,33.489998,25.209999,35.090000,9.880001
26918,28987,2016,281,1526,PLG,1,89,46.549999,0.282121,0.88,0.312105,16.280001,54.619999,38.339998,30.600000,37.299999,6.699999
